# SGG YOLO CONVERTER

### This script can convert any annotations file in SGG format (.h5) in YOLO format for object detection training

In [2]:
import os
import shutil
import random
import h5py
import json
from collections import Counter
from tqdm import tqdm

vg_h5 = h5py.File("/home/maelic/SGG-Benchmark/datasets/vg/VG-SGG-with-attri.h5")
vg_dict = json.load(open("/home/maelic/SGG-Benchmark/datasets/vg/VG-SGG-dicts-with-attri.json"))
image_data = json.load(open('/home/maelic/SGG-Benchmark/datasets/vg/image_data.json'))

img_path = '/home/maelic/Documents/Datasets/VG/VG_100K'

In [6]:
object_to_idx = vg_dict['idx_to_label']

OUT_PATH = "/home/maelic/Documents/Datasets/VG/VG150/YOLO_anno/"
if not os.path.exists(OUT_PATH):
    os.makedirs(OUT_PATH)

with open(os.path.join(OUT_PATH, 'classes.txt'), 'w') as f:
    for key,v in object_to_idx.items():
        f.write(v + '\n')

In [7]:
object_to_idx = vg_dict['label_to_idx']
dest_folder = os.path.join(OUT_PATH, 'annotations')
image_dest_folder = os.path.join(OUT_PATH, 'images_all')

import numpy as np

if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)
if not os.path.exists(image_dest_folder):
    os.makedirs(image_dest_folder)

for idx in tqdm(range(len(vg_h5['img_to_first_box']))):
    img_width = image_data[idx]['width']
    img_height = image_data[idx]['height']
    image_id = image_data[idx]['image_id']
    if vg_h5['img_to_first_box'][idx] == -1 or (vg_h5['img_to_last_box'][idx] == vg_h5['img_to_first_box'][idx]):
        continue
    
    for i in range(vg_h5['img_to_first_box'][idx], vg_h5['img_to_last_box'][idx]):

        obj_cat = vg_h5['labels'][i][0]-1
        assert 0 <= obj_cat < 150
        
        box = vg_h5['boxes_1024'][i]
        # transform coordinates back to original image size
        box[0] = box[0] +1
        box[1] = box[1] +1
        box = box / 1024 * max(img_width, img_height)

        # convert to yolo format
        x_center = box[0] / img_width
        y_center = box[1] / img_height
        w = box[2] / img_width
        h = box[3] / img_height
        
        box = [x_center, y_center, w, h]
        
        # clip to [0, 1]
        box = np.clip(box, 0, 1)
        
        with open(os.path.join(dest_folder, str(image_id) + '.txt'), 'a') as f:
            f.write(str(obj_cat) + ' ' + str(box[0]) + ' ' + str(box[1]) + ' ' + str(box[2]) + ' ' + str(box[3])+'\n')
    
    # copy image to dest folder
    shutil.copyfile(os.path.join(img_path, str(image_id)+'.jpg'), os.path.join(image_dest_folder, str(image_id)+'.jpg'))

100%|██████████| 108073/108073 [41:21<00:00, 43.55it/s]


In [8]:
from sklearn.model_selection import train_test_split

base_path = OUT_PATH
dest_folder = os.path.join(base_path,'annotations')
image_dest_folder = os.path.join(base_path,'images_all')

if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)

if not os.path.exists(image_dest_folder):
    os.makedirs(image_dest_folder)

images = [os.path.join(image_dest_folder, x) for x in os.listdir(image_dest_folder)]
annotations = [os.path.join(dest_folder, x) for x in os.listdir(dest_folder) if x[-3:] == "txt"]

images.sort()
annotations.sort()
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.3, random_state = 1)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

root_path = 'images/'
folders = [base_path+'/train',base_path+'/test',base_path+'/val']
for folder in folders:
    os.makedirs(os.path.join(folder,root_path))
    
root_path = 'labels/'
for folder in folders:
    os.makedirs(os.path.join(folder,root_path))    

def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

move_files_to_folder(train_images, base_path+'/train/images/')
move_files_to_folder(val_images, base_path+'/val/images/')
move_files_to_folder(test_images, base_path+'/test/images/')
move_files_to_folder(train_annotations, base_path+'/train/labels/')
move_files_to_folder(val_annotations, base_path+'/val/labels/')
move_files_to_folder(test_annotations, base_path+'/test/labels/')

In [9]:
# write the yaml file
classes = vg_dict['idx_to_label'].values()

train_path = base_path+'train/'
val_path = base_path+'val/'
test_path = base_path+'test/'

n_classes = len(classes)

with open(base_path+'/train.yaml', 'w') as f:
    f.write('train: '+train_path+'\n')
    f.write('val: '+val_path+'\n')
    f.write('test: '+test_path+'\n')
    f.write('nc: '+str(n_classes)+'\n')
    f.write('names: '+str(list(classes)))